## Importing Libraries

In [1]:
import numpy as np
import pandas as pd

## Importing dataset

In [2]:
dataset = pd.read_csv('../../data/data-6-model.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
print(X[0])
print(y[0])
dataset.head()

[6 'Mathematics' 'Time' 'Games' 49 'Kinesthetic']
Time Difference Board Game


,grade,subject,lesson,class_interests,lesson_average_mark,teaching_aid_category,teaching_aid
0,6,Mathematics,Time,Games,49,Kinesthetic,Time Difference Board Game
1,6,Mathematics,Time,Videos,69,Audio Visual,Time Theory Exposition
2,6,Mathematics,Circles,Science,68,Visual,DIY Compass
3,6,Mathematics,Time,Videos,60,Audio Visual,Time Concepts Animation
4,6,Mathematics,Subtraction,Art,63,Visual,Posters


## Encode Independant variables

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output = False), [1, 2, 3, 5])], remainder='passthrough')
X = ct.fit_transform(X)
print(X[0])

[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 1.0 0.0 6 49]


## Encode dependant variables

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y[0])

91


## Splitting dataset into Test set and Train set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 22:] = sc.fit_transform(X_train[:, 22:])
X_test[:, 22:] = sc.transform(X_test[:, 22:])
print(X_train[0])

[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 1.0 0.0 6 0.9861287363310529]


## Train Random Forest model on the Train set

In [7]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

## Predict the Test set results

In [8]:
y_pred = classifier.predict(X_test)

## Create Confusion Matrix

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
0.8888888888888888


## Applying K-Fold Cross Validation

In [10]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy : {:.2f}".format(accuracies.mean()*100))
print("Standard Deviation : {:.2f}".format(accuracies.std()*100))

Accuracy : 90.22
Standard Deviation : 3.40


/Users/sahanwalpita/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


## Applying GridSearch to find the best hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, None],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
    }]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy : {:.2f}".format(best_accuracy*100))
print("Best parameters : ", best_parameters)